In [ ]:
import papermill as pm
import os
from create_database_functions import get_files, get_averages, make_lists, make_databases, UserClassifier, EEGUserClassifier
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import torch
import pickle
from scipy.spatial.distance import cosine

Get the files

In [ ]:
pm.execute_notebook(
    "C:/Users/Iraklis Kolokas/Documents/GitHub/Diploma-Thesis/EEG_Experiment_Interface.ipynb",
    "C:/Users/Iraklis Kolokas/Documents/GitHub/Diploma-Thesis/EEG_Experiment_Interface.ipynb"
    )

Add parameters to pass file name

In [ ]:
file_path = "C:/Users/Iraklis Kolokas/Desktop/Thesis/Real_time_authentication"

Preprocessing

In [ ]:
#Keep only bdf files",
files =[f for f in os.listdir(file_path) if f.endswith(".bdf")]
                                                                                  
for filename in files:
    filename = filename[:-4]
    #Do the preprocessing for each file
    pm.execute_notebook(
        #input notebook
        f"C:/Users/Iraklis Kolokas/Documents/GitHub/Diploma-Thesis/Real_Time_Pre.ipynb",
        #output notebook
        f"C:/Users/Iraklis Kolokas/Desktop/Thesis/papermill outputs/output_{filename}.ipynb",
        #parameters
        parameters = {"filename": filename}
    )

Get the features from the file

In [ ]:
all_data = get_files(file_path)

Compute the averages per session and per stimuli frequency per user user for each feature extraction model

In [ ]:
av_per_session_tested, freq_av_tested = get_averages(all_data)

Put the data to lists to use with the Neural Network models 

In [ ]:
features_1, stimuli_1, user_ids_1 = make_lists(all_data, 1)
features_2, stimuli_2, user_ids_2 = make_lists(all_data, 2)

In [ ]:
x_1, x_2, y_1, y_2, loader_1, loader_2 = make_databases(features_1, features_2, user_ids_1, user_ids_2)

In [ ]:
model1 = UserClassifier(x_1.shape[1], len(torch.unique(y_1)))
model2 = UserClassifier(x_2.shape[1], len(torch.unique(y_2)))

Add the stimuli labels to the data

In [ ]:
#Add the stimuli labels to each feature with OneHotEncoder
stimuli_1 = np.hstack(stimuli_1)
stimuli_2 = np.hstack(stimuli_2)

stimuli_1 = stimuli_1.reshape(-1,1)
stimuli_2 = stimuli_2.reshape(-1,1)

encoder = OneHotEncoder(sparse_output = False)
stimuli_1 = encoder.fit_transform(stimuli_1)
stimuli_2 = encoder.fit_transform(stimuli_2)

#Add the encoded stimuli labels to the corresponding feature
features_1 = np.hstack([features_1, stimuli_1])
features_2 = np.hstack([features_2, stimuli_2])

In [ ]:
x_3, x_4, y_3, y_4, loader_3, loader_4 = make_databases(features_1, features_2, user_ids_1, user_ids_2)

In [ ]:
model3 = EEGUserClassifier(x_3.shape[1], len(torch.unique(y_3)))
model4 = EEGUserClassifier(x_4.shape[1], len(torch.unique(y_4)))

Load the trained models from their files

In [ ]:
model1.load_state_dict(torch.load("C:/Users/Iraklis Kolokas/Desktop/Thesis/Databases/user_classifier_1.pth"))
model2.load_state_dict(torch.load("C:/Users/Iraklis Kolokas/Desktop/Thesis/Databases/user_classifier_2.pth"))
model3.load_state_dict(torch.load("C:/Users/Iraklis Kolokas/Desktop/Thesis/Databases/user_classifier_w_labels_1.pth"))
model4.load_state_dict(torch.load("C:/Users/Iraklis Kolokas/Desktop/Thesis/Databases/user_classifier_w_labels_2.pth"))

In [ ]:
model1.eval()
model2.eval()
model3.eval()
model4.eval()

Load the averages 

In [ ]:
with open("C:/Users/Iraklis Kolokas/Desktop/Thesis/Databases/each_freq_av.pkl", "rb") as f:
    each_freq_av = pickle.load(f)

with open("C:/Users/Iraklis Kolokas/Desktop/Thesis/Databases/av_per_user.pkl", "rb") as f:
    av_per_user = pickle.load(f)

Cosine similarity

In [ ]:
per_session_sim_1 = {}
per_session_sim_2 = {}

#Look through every user and compare the average with the new users average
for (tested_user_id, tested_model_id), tested_avg in av_per_session_tested.items():
    for (user_id, model_id), avg_features in av_per_user.items():
        if model_id == tested_model_id:   
            sim = 1 - cosine(tested_avg, avg_features)
            if model_id == 1:
                per_session_sim_1[user_id] = sim
            elif model_id == 2:
                per_session_sim_2[user_id] = sim

In [ ]:
per_freq_sim_1 = {}
per_freq_sim_2 = {}

#Look through every user per stimuli label and compare with the new user average per stimuli label
for (tested_user_id, tested_cl_num, tested_model_id), tested_avg in freq_av_tested.items():
    for (user_id, cl_num, model_id), avg_features in each_freq_av.items():
        if tested_cl_num == cl_num and model_id == tested_model_id:    
            sim = 1 - cosine(tested_avg, avg_features)
            if model_id == 1:
                per_freq_sim_1[(user_id, cl_num)] = sim
            elif model_id == 2:
                per_freq_sim_2[(user_id, cl_num)] = sim

In [ ]:
#Print the results and add certainty threshold

Neural Network 

In [ ]:
#Run each model with the correct data to authenticate
with torch.no_grad():
    outputs_1 = model1(x_1)
_, prediction_1 = torch.max(outputs_1, dim = 1)

with torch.no_grad():
    outputs_2 = model2(x_2)
_, prediction_2 = torch.max(outputs_2, dim = 1)

with torch.no_grad():
    outputs_3 = model3(x_3)
_, prediction_3 = torch.max(outputs_3, dim = 1)

with torch.no_grad():
    outputs_4 = model4(x_4)
_, prediction_4 = torch.max(outputs_4, dim = 1)

In [ ]:
#Print the results and add certainty threshold